In [1]:
# disable future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# safe imports
import os, sys       # arcgis comes with these
import datetime      # arcgis comes with this
import numpy as np   # arcgis comes with this
import pandas as pd  # arcgis comes with this
import matplotlib.pyplot as plt

# risk imports (non-native to arcgis)
try:
    import xarray as xr  # not in arcgis
except:
    arcpy.AddError('Python library Xarray is not installed.')
    raise

# import tools
try:
    # shared folder
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\shared')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
    import arc, satfetcher, tools  

    # module folder
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\modules')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
    import ensemble, cog  

except:
    arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
    raise

In [4]:
in_nc = r"C:\Users\262272G\Desktop\temp\sdm.nc"
ds = xr.open_dataset(in_nc)
#ds = ds.compute()

In [13]:
ds_attrs = ds.attrs



'EPSG:3577'

In [15]:
def check_ensemble_layers(in_lyrs):
    
    
    #loop each item in list
    for lyr in in_lyrs:
    
        # get crs, res
        try:
            crs = tools.get_xr_crs(ds)
            res = tools.get_xr_resolution(ds)
        except:
            raise ValueError('Input: {} has no crs/res attributes.'.format(lyr))
            
        # check if crs
    
    
    

30.0

In [4]:
ds_mask = satfetcher.load_local_rasters(rast_path_list=r"C:\Users\262272G\Desktop\south_area\topo_vars\rad.tif", 
                                        use_dask=True, 
                                        conform_nodata_to=np.nan)

ds_mask = ds_mask.to_array()

Converting rasters to an xarray dataset.
Converted raster to xarray data array: rad
Rasters converted to dataset successfully.



<xarray.Dataset>
Dimensions:  (x: 433, y: 445)
Coordinates:
  * y        (y) float64 -2.526e+06 -2.526e+06 ... -2.539e+06 -2.539e+06
  * x        (x) float64 -1.302e+06 -1.302e+06 ... -1.29e+06 -1.29e+06
Data variables:
    rad      (y, x) float32 dask.array<chunksize=(445, 433), meta=np.ndarray>
Attributes:
    transform:      (30.0, 0.0, -1302495.0, 0.0, -30.0, -2525985.0)
    crs:            +init=epsg:3577
    res:            (30.0, 30.0)
    is_tiled:       1
    nodatavals:     nan
    scales:         (1.0,)
    offsets:        (0.0,)
    descriptions:   ('Band_1',)
    AREA_OR_POINT:  Area
    DataType:       Scientific

In [20]:
in_add_result_to_map = True
out_nc = r'C:\Users\262272G\Desktop\test\pheno_novardim.nc'

In [21]:
GRP_LYR_FILE = r"C:\Users\262272G\Documents\GitHub\tenement-tools\arc\lyr\group_template.lyrx"

In [101]:
# # # # #
# add multi-dim raster to current map
if in_add_result_to_map:

    # notify and increment progess bar
    arcpy.SetProgressorLabel('Adding metrics to current ArcGIS map...')
    arcpy.SetProgressorPosition(16)

    # create output folder with dt
    dt = datetime.datetime.now().strftime("%d%m%Y%H%M%S")
    out_folder = os.path.join(os.path.dirname(out_nc), 'metrics' + '_' + dt)
    os.makedirs(out_folder)
    
    try:
        # try to get current map, fail if doesnt exist
        aprx = arcpy.mp.ArcGISProject('CURRENT')
        m = aprx.activeMap
        
        # enable auto-add to map for second
        arcpy.env.addOutputsToMap = True
        
        # setup a group layer via template
        grp_lyr = arcpy.mp.LayerFile(GRP_LYR_FILE)
        grp = m.addLayer(grp_lyr)[0]
        grp.name = 'metrics'

        # loop each metric and export a seperate crf
        for var in list(ds.data_vars): 
            
            # enable auto-add to map
            arcpy.env.addOutputsToMap = False

            # create current output crf file
            out_crf = os.path.join(out_folder, var + '.crf')
                        
            # select current var from crf
            lyr = arcpy.md.MakeMultidimensionalRasterLayer(in_multidimensional_raster=out_nc, 
                                                           out_multidimensional_raster_layer=out_crf,
                                                           variables=[var])
            
            # enable auto-add to map
            arcpy.env.addOutputsToMap = False
            
            # save crf into memory
            crf = arcpy.management.CopyRaster(in_raster=lyr, 
                                              out_rasterdataset=out_crf)
            
            m.addDataFromPath(crf) 

            # determine optimal cmap based on data type 
            if 'value' in var:
                cmap, cutoff = 'Precipitation', 0.5
            else:
                cmap, cutoff = 'Temperature', 0.0
                
            raise

            # apply a colormap
            lyr = arc.apply_cmap(aprx=aprx, 
                                 lyr_name='{}.crf'.format(var),
                                 cmap_name=cmap,
                                 cutoff_pct=cutoff)
            
            
            
            # rename lyr, add to group, remove second instance
            lyr.name = var     
            m.addLayerToGroup(grp, lyr)
            m.removeLayer(lyr)
                                    
            raise
            

    except Exception as e:
        print(e)
        arcpy.AddMessage(e)
        arcpy.AddWarning('Could not visualise var: {}.'.format(var))


No active exception to reraise


In [69]:
arcpy.management.CopyRaster(in_raster=lyr, out_rasterdataset=r'C:\Users\262272G\Desktop\test\huh2.crf')

<Result 'C:\\Users\\262272G\\Desktop\\test\\huh2.crf'>

In [65]:
arcpy.CopyRaster_management(out, 'in_memory/heya.crf')

ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failed to execute (CopyRaster).
